In [ ]:
import pandas as pd
import numpy as np
import os

from pytradersan import constants
from pytradersan.helpers import standardize_transactions
from pytradersan.portfolio import Portfolio
from pytradersan.schwab_api import (
    SCHWAB_API_BASE_URL,
    get_combined_transactions,
    process_raw_trades,
)

from pathlib import Path
import glob
from collections import defaultdict
import requests
import json
from tzlocal import get_localzone
import yfinance as yf

from dotenv import load_dotenv, find_dotenv

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
load_dotenv(find_dotenv())
SCHWAB_BEARER_TOKEN = os.getenv("SCHWAB_BEARER_TOKEN")

In [ ]:
# Get file paths
top_level = Path('../..')
schwab_files = glob.glob(str(top_level / 'data' / '*schwab*.csv'))
marcus_file = glob.glob(str(top_level / 'data' / 'marc*.csv'))[0]
print(f'Schwab files: {schwab_files}')

In [ ]:
sorted([x for x in schwab_files if 'api_trades' in x])

In [ ]:
# Get Schwab transactions

def get_schwab_transactions(schwab_files, use_api=False):
    if use_api:
        transactions = get_combined_transactions(
            base_url=SCHWAB_API_BASE_URL,
            token=SCHWAB_BEARER_TOKEN,
            start_date='2024-01-01',
        )
        raw_schwab_trades = transactions['TRADE'].copy(deep=True)
        raw_schwab_trades['transferItems'] = raw_schwab_trades['transferItems'].apply(json.dumps)
        raw_schwab_trades.to_csv(top_level / 'data' / f'raw_schwab_api_trades_{pd.Timestamp.now().strftime('%Y%m%d')}.csv', index=False)
    else:
        latest_schwab_transactions_file = sorted(
            [x for x in schwab_files if 'api_trades' in x],
            reverse=True
        )[0]
        raw_schwab_trades = pd.read_csv(latest_schwab_transactions_file)
    raw_schwab_trades['transferItems'] = raw_schwab_trades['transferItems'].apply(json.loads)
    return raw_schwab_trades

In [ ]:
# Get Transactions from local path

# Read transactions from Schwab files
raw_schwab_trades = get_schwab_transactions(schwab_files, use_api=False)

# Read transactions from Marcus file
raw_marcus_transactions = pd.read_csv(marcus_file)

In [ ]:
# Process transactions

# Get Marcus trades
marcus_transactions_processed = standardize_transactions(
    'marcus', 'marc8202', raw_marcus_transactions
)
marcus_trades = marcus_transactions_processed[
    marcus_transactions_processed['action'].isin(['BUY', 'SELL'])
].copy(deep=True)

# Get Schwab trades
schwab_trades = process_raw_trades(raw_schwab_trades)

In [ ]:
schwab_trades is not None

In [ ]:
# # Combine all trades
# combined_trades = pd.concat([marcus_trades, schwab_trades], axis=0, ignore_index=True)
# combined_trades['date'] = pd.to_datetime(combined_trades['date']).dt.date
# combined_trades
# prices = yf.download(
#     tickers=combined_trades.symbol.unique().tolist()[:5],
#     period='max',
#     actions=True,
# )

In [ ]:
schwab_portfolio = Portfolio(trades=schwab_trades)
marcus_portfolio = Portfolio(trades=marcus_trades)

In [ ]:
# Testing the Ticker object from yfinance

meta = yf.Ticker('META')
meta_prices = meta.history()
meta_price_targets = meta.analyst_price_targets
meta_calendar = meta.calendar

In [ ]:
meta_calendar